In [1]:
#Import statements
import json
import requests
import pandas as pd
import numpy as np
import datetime
from pandas.io.json import json_normalize
import os
import csv
import time

In [2]:
# The stock Symbols that we will iterate through
# Apple = AAPL
# Amazon = AMZN
# Google = GOOGL
# Microsoft = MSFT
# Dell = DELL
# IBM = IBM
# Intel = INTC
# HP = HPQ
# Facebook = FB
# Cisco Systems = CSCO
# Oracle = ORCL
# HP Enterprise = HPE
# Micron Tech = MU
# DXC Tech = DXC  We have recovered all available historical data March 2017 was IPO
# Thermo Fisher Scientific = TMO

# Qualcomm = QCOM
# Jabil = JBL
# Broadcom = AVGO
# Western Digital = WDC
# Applied Materials = AMAT


# Tesla = TSLA
# Alibaba = BABA
# Nvidia = NVDA
# Walmart = WMT
# Walt Disney = DIS
stockSymbol = ["AAPL", "AMZN", "GOOGL","MSFT", "DELL", "IBM", "INTC", "HPQ",
               "FB", "CSCO", "ORCL", "HPE", "MU", "DXC", "TMO"]

#Change the 

In [3]:
#Gathers the json file and all needed data values.
def collect_Twits(res):
    try:
        df = (json_normalize(res['messages'])[{'id','body','created_at','entities.sentiment.basic','symbols'}])
        
        #Reorders the columns
        df = df[['id','body','created_at','entities.sentiment.basic','symbols']]
        
        #Renames the columns
        df = df.rename(columns = {'created_at':'created', 'entities.sentiment.basic': 'tag'})
        
    except:
        df = (json_normalize(res['messages'])[{'id','body','created_at','entities.sentiment','symbols'}])
        #Reorders the columns
        df = df[['id','body','created_at','entities.sentiment','symbols']]
        
        #Renames the columns
        df = df.rename(columns = {'created_at':'created', 'entities.sentiment': 'tag'})

    
    #Replaces the NAN with a string "none"
    df = df.replace(np.nan, 'none', regex=True)
    
    #Replaces the string as a datetime variable
    dateFormat = "%Y-%m-%dT%H:%M:%SZ"
    df['created'] = pd.to_datetime(df['created'], format=dateFormat)
    
    #returns the dataframe in correct format
    return df

In [ ]:
while True:
    for symbol in stockSymbol:
    
        #selects the file to add to
        file = '{}_twits.csv'.format(symbol)
    
        historicalTwits = pd.read_csv(file)
        oldestID = historicalTwits['id'].iloc[-1]
    
        url = "https://api.stocktwits.com/api/2/streams/symbol/{}.json".format(symbol)
        response = requests.get(url, params = {'max' :  oldestID}).json()
        
        if response['response']['status'] == 429:
            print("requests exceeded")
            time.sleep(360)
        
        if not (response['cursor']['more']):
            stockSymbol.remove(symbol)
            continue
    
        tempTwitsDf = collect_Twits(response)

        newHistoricalTwits = historicalTwits.append(tempTwitsDf)
    
        newHistoricalTwits.to_csv(r'{}_twits.csv'.format(symbol), index=False)
        time.sleep(15)
        newHistoricalTwits.to_csv(r'{}_twitsCopy.csv'.format(symbol), index=False)
        time.sleep(15)

In [7]:
#start the files and gather most recent data right now

#Do not RUN anymore!!

#for symbol in stockSymbol:
    
    #selects the file to add to
    #file = '{}_twits.csv'.format(symbol)
    
    #historicalTwits = pd.read_csv(file)
    #oldestID = historicalTwits['id'][-1]
    
    #url = "https://api.stocktwits.com/api/2/streams/symbol/{}.json".format(symbol)
    #response = requests.get(url, params = {'max' :  oldestID}).json()
    
    #response = requests.get(url).json()
    
    #tempTwitsDf = collect_Twits(response)
    
    #tempTwitsDf.to_csv(r'{}_twits.csv'.format(symbol), index=False)

In [26]:
(response['cursor']['more'])

True

In [8]:
response

{'response': {'status': 200},
 'symbol': {'id': 864,
  'symbol': 'AMZN',
  'title': 'Amazon.com Inc.',
  'aliases': [],
  'is_following': False,
  'watchlist_count': 243772},
 'cursor': {'more': True, 'since': 192649044, 'max': 192648910},
 'messages': [{'id': 192649044,
   'body': '$AMZN woooow',
   'created_at': '2020-01-30T21:04:40Z',
   'user': {'id': 1434549,
    'username': 'stockyappleseed',
    'name': 'Stocky',
    'avatar_url': 'https://avatars.stocktwits.com/production/1434549/thumb-1523824365.png',
    'avatar_url_ssl': 'https://avatars.stocktwits.com/production/1434549/thumb-1523824365.png',
    'join_date': '2018-03-06',
    'official': False,
    'identity': 'User',
    'classification': [],
    'followers': 73,
    'following': 19,
    'ideas': 3363,
    'watchlist_stocks_count': 98,
    'like_count': 3528,
    'plus_tier': '',
    'premium_room': '',
    'trade_app': False},
   'source': {'id': 1149,
    'title': 'StockTwits for iOS',
    'url': 'http://www.stocktwits.

In [15]:
if not (response['cursor']['more']):
    print("Hello")